In [50]:
from cobra.io import read_sbml_model
from collections import defaultdict
from Bio import Entrez, SeqIO
import re
import time
from Bio import pairwise2

model = read_sbml_model('iCGB21FR.xml')

SBML package 'layout' not supported by cobrapy, information is not parsed
https://juser.fz-juelich.de/record/188973 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


The issue with the model "iCGB21FR.xml" is that its authors have named the genes in the following format **"lcl_{locus}_prot_{protein_id}_{unknown_digits}"**. An exemple name of the genes from the model would look like this: lcl_NC_006958_1_prot_WP_003859586_1_2138

In which **"NC_006958"** is the locus represent the complete sequence of Corynebacterium glutamicum ATCC 13032, and **WP_XXXXXXXXX** represent the protein_id of its genes.


**This results in adding an extra step of finding the gene name for the concerned protein ids.**


# Mapping:
- reaction -> genes
- gene -> reactions

In [51]:
print(f"{len(model.reactions)} reactions in 'iCGB21FR' model")
print(f"{len(model.genes)} genes in 'iCGB21FR' model")

1539 reactions in 'iCGB21FR' model
805 genes in 'iCGB21FR' model


In [52]:
reaction_to_genes = {}
gene_to_reactions = {}

def mapReactionToGenes():    
    for reaction in model.reactions:
        gene_ids = []
        for gene in reaction.genes:
            gene_ids.append(gene.id)
            
        reaction_to_genes[reaction.id] = gene_ids

def mapGeneToReactions():
    for reaction_id, gene_ids in reaction_to_genes.items():
        for gene_id in gene_ids:
            if gene_id:
                if gene_id not in gene_to_reactions:
                    gene_to_reactions[gene_id] = [reaction_id]
                else:
                    gene_to_reactions[gene_id].append(reaction_id)
    
mapReactionToGenes()
mapGeneToReactions()

In [53]:
print(f"{len(reaction_to_genes)} reactions are mapped in total")

reactions_coded_by_genes = 0
for reaction, genes in list(reaction_to_genes.items()):
    if len(genes) > 0:
        reactions_coded_by_genes +=1
print(f"{reactions_coded_by_genes} reactions have a genetic component (coded by at least one gene)\n")

print("   ...")
for reaction, genes in list(reaction_to_genes.items())[15:25]:
    print(f"{reaction} -> ")
    for gene in genes:
        print(f"    {gene}")
print("   ...")

1539 reactions are mapped in total
1077 reactions have a genetic component (coded by at least one gene)

   ...
2AGPG141tipp -> 
2AGPGAT141 -> 
2HH24DDH -> 
2MAHMP -> 
    lcl_NC_006958_1_prot_WP_003855288_1_2949
    lcl_NC_006958_1_prot_WP_011015468_1_2761
2PGLYCt6 -> 
3HAD140 -> 
3MBt2pp -> 
    lcl_NC_006958_1_prot_WP_011013917_1_810
3MBt4pp -> 
    lcl_NC_006958_1_prot_WP_011013917_1_810
3MBtex -> 
3OADPCOAT -> 
   ...


In [54]:
print(f"\n{len(gene_to_reactions)} genes are mapped in total\n")

for gene, reactions in list(gene_to_reactions.items())[:5]:
    print(f"{gene} ->")
    for reaction in reactions:
        print(f"    {reaction}")
print("   ...")


805 genes are mapped in total

lcl_NC_006958_1_prot_WP_003855288_1_2949 ->
    2MAHMP
    TDP
lcl_NC_006958_1_prot_WP_011015468_1_2761 ->
    2MAHMP
    PDXPP
    PYDXPP
lcl_NC_006958_1_prot_WP_011013917_1_810 ->
    3MBt2pp
    3MBt4pp
    ACt2rpp
    ACt4pp
    BUTt2rpp
    BUTt4pp
    GLYCLTt4pp
    PACt3
    PNTOt2
    PPAt2pp
    PPAt4pp
    PYRt2rpp
    PYRt4pp
lcl_NC_006958_1_prot_WP_003859251_1_2275 ->
    3OXCOAT
    ACACT1r
    ACACT2r
lcl_NC_006958_1_prot_WP_011015386_1_2669 ->
    4ABUTD
    ALDD2x
    ALDD2y
    ALDD31
    ALDD3y
    GLXO1
    GLYALDDr
    LCADi
   ...


In [55]:
gene_to_reactions_counts = defaultdict(list)

for gene, reactions in gene_to_reactions.items():
    gene_to_reactions_counts[len(reactions)].append((gene, reactions))

sorted_counts = sorted(gene_to_reactions_counts.items(), key=lambda x: x[0])

for count, gene_to_reactions_data in sorted_counts:
    reaction_str = "reaction" if count == 1 else "reactions"
    print(f"{len(gene_to_reactions_data)} genes control {count} {reaction_str}")

513 genes control 1 reaction
149 genes control 2 reactions
51 genes control 3 reactions
25 genes control 4 reactions
12 genes control 5 reactions
19 genes control 6 reactions
6 genes control 7 reactions
4 genes control 8 reactions
5 genes control 9 reactions
5 genes control 10 reactions
4 genes control 11 reactions
4 genes control 13 reactions
1 genes control 14 reactions
2 genes control 15 reactions
1 genes control 21 reactions
1 genes control 22 reactions
3 genes control 23 reactions


### Getting the gene data from the RefSeq database based on its protein ID

Fetching every protein Id in **gene_to_reactions** and returning the data for its gene: gene_name, product, organism, sequence.

The data is stored locally in **protein_id_to_gene_map** for future use, avoiding searching every time.

In [56]:
import re
from Bio import Entrez, SeqIO
import time
import json

def getProteinID(full_gene):
    """
    Function to get protein ID from a full gene name
    
    """
    return "WP_" + re.search(r"WP_(\d{9})", full_gene).group(1) if re.search(r"WP_(\d{9})", full_gene) else None


def fetchProteinID(protein_id, max_retries=50, delay_between_retries=1):
    """
    Function to fetch protein ID data
    
    """
    for attempt in range(max_retries):
        try:
            Entrez.email = "h.bensaadi@gmail.com"

            handle = Entrez.efetch(db="protein", id=protein_id, rettype="gb", retmode="text")
            record = SeqIO.read(handle, "genbank")
            handle.close()

            gene_name, product, organism, sequence = None, None, None, None

            for feature in record.features:
                if 'gene' in feature.qualifiers:
                    gene_name = feature.qualifiers['gene'][0]
                if 'product' in feature.qualifiers:
                    product = feature.qualifiers['product'][0]
                if 'organism' in feature.qualifiers:
                    organism = feature.qualifiers['organism'][0]

            sequence = str(record.seq)

            return protein_id, gene_name, product, organism, sequence

        except Entrez.HTTPError as e:
            if attempt < max_retries - 1:
                print(f"{protein_id}: retrying in {delay_between_retries} seconds...")
                time.sleep(delay_between_retries)
            else:
                print(f"Max retries reached. Unable to fetch data for {protein_id}")
                raise
        except Exception as e:
            print(f"Error: {e}")
            raise

def loadMapFile():
    try:
        with open("protein_id_to_gene_map.json", "r") as file:
            protein_id_to_gene_map = json.load(file)
    except FileNotFoundError:
        protein_id_to_gene_map = {}
    
    return protein_id_to_gene_map
    
def updateMapFile(protein_id_to_gene_map):
    with open("protein_id_to_gene_map.json", "w") as file:
        json.dump(protein_id_to_gene_map, file)
           
            
def getUniqueGeneNumber():
    unique_protein_ids = []
    
    for gene in list(gene_to_reactions.items()):
        if getProteinID(gene) not in unique_protein_ids:
            unique_protein_ids.append(getProteinID(gene))
            
    return len(unique_protein_ids)
    

def mapProteinIDtoGene():

    print(f"{getUniqueGeneNumber()} protein ids to fetch its data")
    
    for gene, reactions in list(gene_to_reactions.items()):
        if "WP_" in gene:
            protein_id = getProteinID(gene)

            if protein_id not in protein_id_to_gene_map:
                protein_id, gene_name, product, organism, sequence = fetchProteinID(protein_id)
                protein_id_to_gene_map[protein_id] = [gene_name, product, organism, sequence]
                
                updateMapFile(protein_id_to_gene_map)
                
                print(f"\n{protein_id}, {gene_name}, {product}, {organism}, {sequence}")

    print(f"{count_found_genes} protein IDs were fetched")
    print(f"{len(protein_id_to_gene_map)} protein IDs are stored locally")
    
protein_id_to_gene_map = loadMapFile()
 
mapProteinIDtoGene()

for protein_id, protein_data in list(protein_id_to_gene_map.items())[21:23]:
    print(f"\n{protein_id} ->")
    gene_name, product, organism, sequence = protein_data
    print(f"    {gene_name}")
    print(f"    {product}")
    print(f"    {organism}")
    print(f"    {sequence[:70]}...")
print("   ...")

TypeError: expected string or bytes-like object

In [ ]:
unique_protein_ids = []

for gene, reactions in list(gene_to_reactions.items()):
    print(gene)
    if getProteinID(gene) not in unique_protein_ids:
        unique_protein_ids.append(getProteinID(gene))

return len(unique_protein_ids)


### Serching the protein ID of a known gene 
The locally stored **protein_id_to_gene_map** is used to search for the **gene name** or its **sequence**

In [ ]:
def calculate_similarity(seq1, seq2):
    alignments = pairwise2.align.globalxx(seq1, seq2, one_alignment_only=True)
    if alignments:
        alignment = alignments[0]
        alignment_length = max(len(alignment[0]), len(alignment[1]))
        similarity = alignment[2] / alignment_length
        return similarity
    return 0

def searchGene(gene=None, metabolite=None, seq=None, similarity_threshold=0.80):
    matches = []
    
    protein_id_to_gene_map = loadMapFile()

    for protein_id, data in protein_id_to_gene_map.items():
        gene_name, product, organism, sequence = data
        
        if gene and gene_name and gene.lower() in gene_name.lower():
            matches.append((protein_id, data))
        
        if metabolite and product and metabolite.lower() in product.lower():
            matches.append((protein_id, data))
        
        
        if seq and sequence:
            similarity = calculate_similarity(seq.lower(), sequence.lower())            
            if similarity >= similarity_threshold:
                matches.append((protein_id, f"Similarity: {round(similarity*100, 2)}%", data))
    
    print(f"{len(matches)} match(es) were found:")
    return matches

def getDataFromProtreinID(searched_protein_id):
    protein_id_to_gene_map = loadMapFile()
    for protein_id, data in protein_id_to_gene_map.items():
        if searched_protein_id == protein_id:
            return data

def getGeneNameFromProteinID(searched_protein_id):
    for gene, reactions in list(gene_to_reactions.items()):
        if searched_protein_id == getProteinID(gene):
            return gene
    
def getReactionsFromProteinID(searched_protein_id):
    for gene, reactions in list(gene_to_reactions.items()):
        if searched_protein_id == getProteinID(gene):
            return reactions
    

## Gene knockout:

Replicating the knockout of 4 genes from https://link.springer.com/article/10.1007/s00726-020-02893-6 
- gluE
- brnE
- brnF
- lysP

In [57]:
genes = ["gluE", "brnE", "brnF", "lysP"]

sequences = []
# gluE
sequences.append("")
# brnE
sequences.append("")
# brnF
sequences.append("")
# lysP
sequences.append("")


for gene in genes:
    searchGene(gene=gene)
    
for sequence in sequences:
    searchGene(seq=sequence, similarity_threshold=0.60)

0 match(es) were found:
0 match(es) were found:
0 match(es) were found:
0 match(es) were found:
0 match(es) were found:
0 match(es) were found:
0 match(es) were found:
0 match(es) were found:


### Conclusion:
The genes used in model "iCGB21FR" contanins that are not annotated, at least for the genes that we want to knockout (gluE, brnE, brnF and lysP).

**Solution**: we need to manually find the gene name by finding the reaction through searching metabolites

## Knocking out

### 1. lysP

In [77]:
genes = {"lysP": "lcl_NC_006958_1_prot_WP_003858826_1_1288"}

In [69]:
searched_protein_id = getProteinID(genes["lysP"])

In [78]:
gene_name, product, organism, sequence = getDataFromProtreinID(searched_protein_id)
gene = getGeneNameFromProteinID(searched_protein_id)
reactions = getReactionsFromProteinID(searched_protein_id)

print(f"Gene:\n   - {gene_name}\nProduct:\n  - {product}\nOrganism:\n  - {organism}\nSequence:\n  - {sequence[:75]}...")
print(f"\nFull name of the gene in the model: \n    {gene}")
print(f"\nControlled reactions:")
for reaction in reactions:
    print(f"   - {model.reactions.get_by_id(reaction)}")

Gene:
   - None
Product:
  - amino acid ABC transporter ATP-binding protein
Organism:
  - Corynebacterium
Sequence:
  - MSQLMIDAQQVCKNYGRLEVLKGIDLQVPKGTVTCLIGPSGSGKSTMLRCVNHLEKVNAGRLYVDGDLIGYRERD...

Full name of the gene in the model: 
    lcl_NC_006958_1_prot_WP_003858826_1_1288

Controlled reactions:
   - ARGabc: arg__L_e + atp_c + h2o_c --> adp_c + arg__L_c + h_c + pi_c
   - ARGabcpp: arg__L_p + atp_c + h2o_c --> adp_c + arg__L_c + h_c + pi_c
   - GLNabc: atp_c + gln__L_e + h2o_c --> adp_c + gln__L_c + h_c + pi_c
   - GLUabc: atp_c + glu__L_e + h2o_c --> adp_c + glu__L_c + h_c + pi_c
   - HISabc: atp_c + h2o_c + his__L_e --> adp_c + h_c + his__L_c + pi_c
   - HISabcpp: atp_c + h2o_c + his__L_p --> adp_c + h_c + his__L_c + pi_c
   - LYSabc: atp_c + h2o_c + lys__L_e --> adp_c + h_c + lys__L_c + pi_c
   - LYSabcpp: atp_c + h2o_c + lys__L_p --> adp_c + h_c + lys__L_c + pi_c
   - ORNabc: atp_c + h2o_c + orn_e --> adp_c + h_c + orn_c + pi_c
   - ORNabcpp: atp_c + h2o_c + orn_p --> adp_c 

In [103]:
results_model = model.optimize()
print('complete model: ', results_model)
with model:
    model.genes.get_by_id("lcl_NC_006958_1_prot_WP_011014457_1_1531").knock_out()
    model.genes.get_by_id("lcl_NC_006958_1_prot_WP_011014763_1_1832").knock_out()
    model.genes.get_by_id("lcl_NC_006958_1_prot_WP_011014747_1_1809").knock_out()
    
    results_model_knocked_out = model.optimize()
    print(f'lcl_NC_006958_1_prot_WP_011014457_1_1531 knocked out: ', results_model_knocked_out)
    

complete model:  <Solution 0.571 at 0x236800ccfa0>
lcl_NC_006958_1_prot_WP_011014457_1_1531 knocked out:  <Solution 0.500 at 0x236fe33b100>


In [104]:
results_model

,fluxes,reduced_costs
12DGR120tipp,0.0,-3.469447e-18
12DGR140tipp,0.0,0.000000e+00
12DGR161tipp,0.0,-3.469447e-18
12DGR180tipp,0.0,-3.469447e-18
12DGR181tipp,0.0,2.775558e-17
...,...,...
EX_pyr_e,0.0,-3.084649e-03
L_LACtex,0.0,2.168404e-19
CYTB1,20.0,4.163336e-17
EX_34dhbz_e,0.0,0.000000e+00


In [105]:
results_model_knocked_out

,fluxes,reduced_costs
12DGR120tipp,0.0,-1.734723e-18
12DGR140tipp,0.0,-1.734723e-18
12DGR161tipp,0.0,-1.734723e-18
12DGR180tipp,0.0,-1.734723e-18
12DGR181tipp,0.0,2.775558e-17
...,...,...
EX_pyr_e,0.0,-1.203880e-03
L_LACtex,0.0,-3.751148e-17
CYTB1,20.0,5.551115e-17
EX_34dhbz_e,0.0,0.000000e+00


In [94]:
results_model["LYSt3pp"]

0.0

In [95]:
results_model_knocked_out["LYSt3pp"]

0.0

In [91]:
model.reactions.DAPDC

Reaction identifier,DAPDC
Name,Diaminopimelate decarboxylase
Memory address,0x236fd68ff40
Stoichiometry,"26dap__M_c + h_c --> co2_c + lys__L_c Meso-2,6-Diaminoheptanedioate + H+ --> CO2 + L-Lysine"
GPR,lcl_NC_006958_1_prot_WP_011014180_1_1145 or lcl_NC_006958_1_prot_WP_011265883_1_2027
Lower bound,0.0
Upper bound,1000.0
